<a href="https://www.kaggle.com/marcorochauribe/hours-challenge?scriptVersionId=89947257" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/sample_submission.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/site_metadata.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_141.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_202.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_26.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_165.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_124.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_178.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_33.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_49.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_79.csv
/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data

In [2]:
engine_metadata = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/engine_metadata.csv")

In [3]:
engine_metadata["PLANT_NAME"].unique()

array(['SPIRITUAL-POLECAT', 'NIFTY-ROOK', 'PREHISTORIC-PETREL',
       'THERAPEUTIC-LIONFISH', 'ABORIGINAL-PICULET', 'CARMINE-REINDEER',
       'IDEALISTIC-DODO', 'ANCIENT-WASP', 'AQUAMARINE-KANGAROO',
       'RADIANT-STURGEON', 'CURIOUS-QUAIL', 'SILKY-WEASEL',
       'INDIGO-TAPIR', 'PERFECT-MACKEREL', 'ADORABLE-PANTHER',
       'CAREFUL-PHEASANT', 'PRECISE-POGONA', 'FIERY-BOBCAT',
       'BIPEDAL-UAKARI', 'LUSH-CHIPMUNK', 'HYPERSONIC-BULLDOG',
       'TUSCAN-CAPUCHIN', 'MUSTARD-PANDA', 'NIFTY-MULE',
       'TOURMALINE-MOUSE', 'ANNOYING-SPONGE', 'ROSE-KINGFISHER',
       'BEAUTIFUL-HIPPO', 'VORACIOUS-PARROT', 'STOIC-KOEL',
       'COBALT-CATFISH', 'CERULEAN-NAUTILUS', 'PRAGMATIC-MASTIFF',
       'RAPID-CARACARA', 'COBALT-MALKOHA', 'SAGE-PRAWN',
       'ANGELIC-REINDEER', 'INESCAPABLE-QUOKKA', 'DIDACTIC-HORNET',
       'SINGING-HONEYBEE', 'LOUD-OCELOT', 'LOUTISH-PANDA', 'ZIRCON-CAMEL',
       'GIGANTIC-EARWIG', 'PREMIUM-MOTH'], dtype=object)

In [4]:
engine_metadata[(engine_metadata["PLANT_NAME"] == "ANCIENT-WASP") & (engine_metadata["ENGINE_ID"] == "ENGINE_3")]

,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID,FILE_ID
32,BUSY,ANCIENT-WASP,ENGINE_3,data_33.csv


In [5]:
Ancient_wasp_engine3 = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/dataset/dataset/data_33.csv")

In [6]:
Ancient_wasp_engine3.head()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2
0,01/01/2021 00:00,5589.869257,4900.793536,0.451669,1.229985
1,01/01/2021 01:00,5586.765793,4879.848427,0.450786,1.227582
2,01/01/2021 02:00,5599.140667,4902.774026,0.451686,1.230033
3,01/01/2021 03:00,5601.607464,4902.992111,0.451230,1.228792
4,01/01/2021 04:00,5573.922031,4878.853341,0.450572,1.227000


In [7]:
sample_submission = pd.read_csv("/kaggle/input/0db0ed38-868c-40d2-bcbd-bcedb88543cf/sample_submission.csv")
sample_submission.tail()

,ID,HOURS
8756,8756,6624
8757,8757,6625
8758,8758,6626
8759,8759,6627
8760,8760,6628


In [8]:
ID = [i for i in range(8761)]
HOURS = []

In [9]:
for i in ID:
    try:
        if (Ancient_wasp_engine3.loc[i, "CMP_SPEED"] > 0):
            if (Ancient_wasp_engine3.loc[i+1, "CMP_SPEED"] > 0):
                try:
                    HOURS.append(HOURS[-1] + 1)
                except IndexError:
                    HOURS.append(0)
            else:
                HOURS.append(HOURS[-1] + 1)
        else:
            HOURS.append(HOURS[-1] + 0)
    except KeyError:
        HOURS.append(HOURS[-1])

In [10]:
HOURS[8760]

6934

In [11]:
df = pd.DataFrame()

In [12]:
df["ID"] = ID
df["HOURS"] = HOURS

df = df.set_index("ID")

In [13]:
df.to_csv("hours_challenge.csv")

In [14]:
view = pd.concat([Ancient_wasp_engine3, df], axis=1)

In [15]:
pd.set_option('display.max_rows', None)

In [16]:
view.iloc[495:530]

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
495,21/01/2021 15:00,5124.705362,4013.477362,0.413487,1.126008,495
496,21/01/2021 16:00,5105.505536,3992.349503,0.412962,1.124578,496
497,21/01/2021 17:00,5087.712927,3960.169250,0.411381,1.120273,497
498,21/01/2021 18:00,5097.181598,3954.176154,0.410678,1.118360,498
499,21/01/2021 19:00,5092.612539,3952.863582,0.411213,1.119816,499
500,21/01/2021 20:00,5088.938479,3961.875200,0.411914,1.121725,500
501,21/01/2021 21:00,5107.311761,3970.134777,0.411855,1.121564,501
502,21/01/2021 22:00,5120.505012,3987.924090,0.412709,1.123891,502
503,21/01/2021 23:00,5092.968057,3945.975558,0.410625,1.118216,503
504,22/01/2021 00:00,5102.613535,3956.523960,0.410913,1.118999,504


In [17]:
view[view["CMP_SPEED"] == 0].head().tail()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
506,22/01/2021 02:00,0.0,NaN,NaN,NaN,505
507,22/01/2021 03:00,0.0,NaN,NaN,NaN,505
508,22/01/2021 04:00,0.0,NaN,NaN,NaN,505
509,22/01/2021 05:00,0.0,NaN,NaN,NaN,505
510,22/01/2021 06:00,0.0,NaN,NaN,NaN,505


In [18]:
view.tail()

,Unnamed: 0,CMP_SPEED,POWER,FUEL_FLOW,CO2,HOURS
8756,31/12/2021 20:00,0.0,NaN,NaN,NaN,6934
8757,31/12/2021 21:00,0.0,NaN,NaN,NaN,6934
8758,31/12/2021 22:00,0.0,NaN,NaN,NaN,6934
8759,31/12/2021 23:00,0.0,NaN,NaN,NaN,6934
8760,01/01/2022 00:00,0.0,NaN,NaN,NaN,6934
